In [1]:
#Проект чат-бот который считает % проверенных домашних заданий педагогами и отправляет автоматическое сообщение педагогу, если % проверки ниже 75%

In [2]:
import telebot #Библеотека для работы с чат-ботом телеграм
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [ ]:
import telebot
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import threading
import time
from datetime import datetime

# Установите свои учетные данные для Google Sheets
SCOPE = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive.file",
    "https://www.googleapis.com/auth/drive"
]

# Замените на путь к вашему файлу credentials.json
CREDENTIALS_FILE = "C:/Users/1/Desktop/Project (Bot)/projectbot-447014-437b4c89d4a6.json"
credentials = ServiceAccountCredentials.from_json_keyfile_name(CREDENTIALS_FILE, SCOPE)

# Подключение к Google Sheets
client = gspread.authorize(credentials)
spreadsheet_name = "Отчёт по дз"
sheet = client.open(spreadsheet_name).sheet1

# Создание бота
bot = telebot.TeleBot('7809768765:AAHOm3QIEj-TEE-XTWZ9_Y8R0Tn0rzK8hwk')

# Ваши Chat ID
MY_CHAT_ID = 699343727  # Ваш Chat ID
CHAT_ID = 366888178  # Другой Chat ID

def send_scheduled_message():
    """
    Поток для автоматической отправки сообщений в заданное время
    """
    target_time = "17:26"  # Замените на нужное время (часы:минуты)
    while True:
        # Текущее время
        now = datetime.now().strftime("%H:%M")
        if now == target_time:
            try:
                # Получение данных из таблицы
                data = sheet.get_all_values()
                
                # Форматирование данных для отправки
                formatted_data = "\n".join(["\t".join(row) for row in data])
                
                # Сообщение для отправки
                message = f"Автоматическое сообщение от бота:\n\n{formatted_data}"
                
                # Отправка сообщений
                bot.send_message(MY_CHAT_ID, message)
                bot.send_message(CHAT_ID, message)
                print(f"[{datetime.now()}] Сообщения отправлены!")
            except Exception as e:
                print(f"[{datetime.now()}] Ошибка при отправке сообщения: {e}")
            
            # Ждём 60 секунд, чтобы избежать повторной отправки в течение той же минуты
            time.sleep(60)
        else:
            # Проверяем каждую секунду
            time.sleep(1)

# Создание и запуск потока
thread = threading.Thread(target=send_scheduled_message, daemon=True)
thread.start()

@bot.message_handler(commands=['start'])
def start(message):
    bot.send_message(message.chat.id, f'Привет, {message.from_user.first_name} {message.from_user.last_name}!')

@bot.message_handler(commands=['report'])
def report(message):

    bot.send_message(MY_CHAT_ID, f'Команду /report выполняет: {message.from_user.first_name} {message.from_user.last_name} {message.from_user.username} {message.from_user.id}')
    #bot.send_message(MY_CHAT_ID, message)
    try:
        # Получение данных из таблицы
        data = sheet.get_all_values()
        
        # Форматирование данных для отправки
        formatted_data = "\n".join(["\t".join(row) for row in data])
        bot.send_message(message.chat.id, f"Данные из таблицы:\n{formatted_data}")
    except Exception as e:
        bot.send_message(message.chat.id, f"Ошибка при чтении данных: {e}")

# Запуск бота
print("Бот запущен. Поток для автоматической отправки сообщений работает...")
bot.polling(none_stop=True)


Бот запущен. Поток для автоматической отправки сообщений работает...
